In [2]:
import numpy as np
import pandas as pd
import xarray as xr
# %pip install cpmpy
import cpmpy as cp
import os
import math

# old

In [24]:
def initial_setup(folder_name = 'Experiments/Experiment 1', model_names = ['CatBoostRegressor'], date = '2024-07-26', size=100):
    inference_graphs_dir = f'{folder_name}/Inference_Graphs'
    all_product_paths = []

    for model_name in model_names:
        base_path = os.path.join(inference_graphs_dir, model_name, date)
        if not os.path.exists(base_path):
            print(f"Path does not exist: {base_path}")
            continue
        
        all_product_paths.extend([(product_id, os.path.join(base_path, product_id)) 
                                  for product_id in os.listdir(base_path) 
                                  if product_id.isdigit()])

    sorted_product_paths = sorted(all_product_paths)
    
    sku_map = pd.read_csv('data/sku_product_name_map.csv')
    national_price_df = pd.read_csv('s3://elektra-data/commercial_comments/national_price_20240722.csv',index_col=0)

    data = {
        'prices': [], 'revenue': [], 'cost': [], 'demand': [],
        'product_ids': [], 'national_price_creditA': [], 'national_price_cashA': [],
        'national_price_DE': [], 'margin': [], 'min_credit_A': [], 'max_credit_A': [],
        'price_segment_cluster': []
    }

    for product_id, product_dir in sorted_product_paths:
        csv_file = os.path.join(product_dir, f'{date}_uncertainty_metrics.csv')
        if not os.path.exists(csv_file):
            continue

        df = pd.read_csv(csv_file)[:size]
        df = df.astype({'Price Credit': int, 'Price Cash': int, 'Credit Prediction': int, 'Credit Pred Revenue': int})

        data['prices'].append([list(pair) for pair in zip(df['Price Credit'], df['Price Cash'])])
        data['revenue'].append(df['Credit Pred Revenue'].tolist())
        data['demand'].append(df['Credit Prediction'].tolist())
        data['cost'].append(abs(df['Cost of Sale Credit']).astype(int).tolist())
        data['product_ids'].append(product_id)

        product_national_price = national_price_df[national_price_df['SKU'] == int(product_id)].iloc[0]
        
        data['national_price_creditA'].append(int(product_national_price['Control Credit Precio A']))
        data['national_price_cashA'].append(int(product_national_price['Control Cash Precio A']))
        data['national_price_DE'].append(int(product_national_price['Control Precio DE ($)']))
        data['min_credit_A'].append(int(df['Price Credit'].min()))
        data['max_credit_A'].append(int(df['Price Credit'].max()))
        data['price_segment_cluster'].append(product_national_price['price_segment'])
        data['margin'].append(int((df['Price Credit'].mean() - df['Cost of Sale Credit'].mean()) / df['Price Credit'].mean()))

    price_pair_options = list(range(1, 101))

    xr_data = {
        'prices_xr': xr.DataArray(data['prices'], dims=["product", "price_pair_option", "price"], 
                               coords={"cluster": ("product", data['price_segment_cluster']),
                                       "product": data['product_ids'],
                                       "price_pair_option": price_pair_options}, name="Prices"),
        'demands_xr': xr.DataArray(data['demand'], dims=["product", "price_pair_option"], 
                                coords={"product": data['product_ids'], "price_pair_option": price_pair_options}, name="Demands"),
        'costs_xr': xr.DataArray(data['cost'], dims=["product", "price_pair_option"], 
                              coords={"product": data['product_ids']}, name="Costs"),
        'revenues_xr': xr.DataArray(data['revenue'], dims=["product", "price_pair_option"], 
                                 coords={"product": data['product_ids'], "price_pair_option": price_pair_options}, name="Revenues"),
        'national_price_creditA_xr': xr.DataArray(data['national_price_creditA'], dims=["product"], 
                                               coords={"product": data['product_ids']}, name="NationalPriceCreditA"),
        'national_price_cashA_xr': xr.DataArray(data['national_price_cashA'], dims=["product"], 
                                             coords={"product": data['product_ids']}, name="NationalPriceCashA"),
        'national_price_DE_xr': xr.DataArray(data['national_price_DE'], dims=["product"], 
                                          coords={"product": data['product_ids']}, name="NationalPriceDE"),
        'min_price_creditA_xr': xr.DataArray(data['min_credit_A'], dims=["product"], 
                                          coords={"product": data['product_ids']}, name="MinPriceCreditA"),
        'max_price_creditA_xr': xr.DataArray(data['max_credit_A'], dims=["product"], 
                                          coords={"product": data['product_ids']}, name="MaxPriceCreditA"),
        'margin_xr': xr.DataArray(data['margin'], dims=["product"], 
                               coords={"product": data['product_ids']}, name="Margins")
    }

    return xr_data

def define_model(xr_data):    
    num_products, num_price_pairs = xr_data['prices_xr'].shape[0], xr_data['prices_xr'].shape[1]
    price_indices = cp.intvar(0, num_price_pairs-1, shape=(num_products,), name='price_indices')
    model = cp.Model()

    add_revenue_objective(model, xr_data['revenues_xr'], price_indices)
    add_constraints(model, xr_data, price_indices)

    return model, price_indices


def add_revenue_objective(model: cp.Model, revenues_xr, price_indices):
    total_revenue = sum(
        cp.Element(revenues_xr.values[p, :], price_indices[p])
        for p in range(revenues_xr.shape[0])
    )
    model.maximize(total_revenue)

def add_constraints(model: cp.Model, xr_data, price_indices):
    for product in xr_data['prices_xr'].product.values:
        product_idx = xr_data['prices_xr'].get_index('product').get_loc(product)
        national_price_creditA = xr_data['national_price_creditA_xr'].sel(product=product).values
        national_price_DE = xr_data['national_price_DE_xr'].sel(product=product).values

        model += (cp.Element(xr_data['prices_xr'].sel(product=product, price=0).values,
                             price_indices[product_idx]) * 100 < national_price_DE * 100)

    price_segments = xr_data['prices_xr'].coords['cluster'].values
    unique_segments = np.unique(price_segments)

    for segment in unique_segments:
        segment_products = xr_data['prices_xr'].where(xr_data['prices_xr'].coords['cluster'] == segment, drop=True)
        segment_margins = xr_data['margin_xr'].where(xr_data['prices_xr'].coords['cluster'] == segment, drop=True)

        sorted_indices = np.argsort(-segment_margins.values)
        sorted_products = segment_products.isel(product=sorted_indices)

        if segment != 10:
            top_30_percent_index = int(len(sorted_products.product) * 0.3)
            top_30_products = sorted_products.isel(product=slice(0, top_30_percent_index))
            remaining_70_products = sorted_products.isel(product=slice(top_30_percent_index, None))

            add_price_constraints(model, xr_data, price_indices, top_30_products, 90, 100)
            add_price_constraints(model, xr_data, price_indices, remaining_70_products, 97, 102)
        else:
            add_price_constraints(model, xr_data, price_indices, sorted_products, 90, 100)

def add_price_constraints(model, xr_data, price_indices, products, lower_percent, upper_percent):
    for product in products.product.values:
        product_idx = xr_data['prices_xr'].get_index('product').get_loc(product)
        national_price_creditA = xr_data['national_price_creditA_xr'].sel(product=product).values
        price_element = cp.Element(xr_data['prices_xr'].sel(product=product, price=0).values,
                                   price_indices[product_idx]) 

        model += (price_element * 100 <= national_price_creditA * upper_percent )
        model += (price_element * 100 >= national_price_creditA * lower_percent )

def solve_model(model: cp.Model, price_indices, xr_data):  
    result = model.solve()
    
    if result:
        solution = pd.DataFrame(columns=['Product','Catalan Credit A','Catalan Cash A','National Credit A','National Cash A','National DE','Min Credit A','Max Credit A','Demand','Revenue'])

        
        print("Optimal total Revenue found:", model.objective_value())
        for p in range(xr_data['prices_xr'].shape[0]):
            price_index = price_indices[p].value()
            best_price_pair = xr_data['prices_xr'][p, price_index, :].values
            credit_A = xr_data['prices_xr'][p, price_index, 0].values
            cash_A = xr_data['prices_xr'][p, price_index, 1].values
            revenue = xr_data['revenues_xr'][p, price_index].values
            demand = xr_data['demands_xr'][p, price_index].values
            # cluster = prices_xr.coords['cluster'].values[p]
            product = xr_data['prices_xr'].coords['product'].values[p]
            natinal_credit_price = xr_data['national_price_creditA_xr'].sel(product=product).values
            natinal_cash_price = xr_data['national_price_cashA_xr'].sel(product=product).values
            min_credit_price = xr_data['min_price_creditA_xr'].sel(product=product).values
            max_credit_price = xr_data['max_price_creditA_xr'].sel(product=product).values
            national_DE_price = xr_data['national_price_DE_xr'].sel(product=product).values
            print(f" Product {product}: Best Price Pair Index {price_index}, Prices: {best_price_pair}, Revenue: {revenue}, Demand: {demand}")
        
            new_row = pd.DataFrame([[
                                        product,
                                        credit_A,
                                        cash_A,
                                        # best_price_pair,
                                        natinal_credit_price,
                                        natinal_cash_price,
                                        national_DE_price,
                                        min_credit_price,
                                        max_credit_price,
                                        demand,
                                        revenue
                                            ]], columns=['Product','Catalan Credit A','Catalan Cash A','National Credit A','National Cash A','National DE','Min Credit A','Max Credit A','Demand','Revenue'])
            solution = pd.concat([solution,new_row], ignore_index=True)
        
        solution.to_csv('Solution4.csv')
        
        
        return solution
    else:
    
        print("No solution found. Performing infeasibility analysis...")
        
        
        return None

def perform_ILP():
    
    prices_xr, demands_xr, costs_xr, revenues_xr = generate_data_xarray(5,4,20)
    model, price_indices = define_model(prices_xr, revenues_xr, costs_xr, demands_xr)
    
    solution_df = solve_model(model, price_indices, prices_xr, revenues_xr)
    return solution_df

# prices_xr, demands_xr, costs_xr, revenues_xr, national_price_creditA_xr, national_price_cashA_xr, national_price_DE_xr, min_price_creditA_xr, max_price_creditA_xr, margin_xr = initial_setup()
xr_data = initial_setup(folder_name = 'Experiments/Experiment 1', model_names = ['CatBoostRegressor'], date = '2024-07-26', size=100)
model, price_indices = define_model(xr_data)

# model, price_indices = define_model(xr_data['prices_xr'], xr_data['revenues_xr'], xr_data['costs_xr'], xr_data['demands_xr'],xr_data['national_price_creditA_xr'], xr_data['national_price_cashA_xr'], xr_data['national_price_DE_xr'], xr_data['margin_xr'],xr_data)
# solution_df = solve_model(model, price_indices, xr_data['prices_xr'], xr_data['revenues_xr'],xr_data['demands_xr'],xr_data['national_price_creditA_xr'], xr_data['national_price_cashA_xr'], xr_data['min_price_creditA_xr'], xr_data['max_price_creditA_xr'],xr_data['national_price_DE_xr'])
solution_df = solve_model(model, price_indices, xr_data)


Optimal total Revenue found: 790507
 Product 31030392: Best Price Pair Index 50, Prices: [1080 1080], Revenue: 750, Demand: 1
 Product 31032647: Best Price Pair Index 20, Prices: [1774 1774], Revenue: 1213, Demand: 2
 Product 31036667: Best Price Pair Index 57, Prices: [1020 1020], Revenue: 3268, Demand: 8
 Product 31041272: Best Price Pair Index 56, Prices: [1221 1221], Revenue: 305, Demand: 1
 Product 31043249: Best Price Pair Index 17, Prices: [783 783], Revenue: 0, Demand: 0
 Product 31044050: Best Price Pair Index 50, Prices: [2700 2700], Revenue: 500, Demand: 1
 Product 31044759: Best Price Pair Index 41, Prices: [2189 2189], Revenue: 0, Demand: 0
 Product 31044766: Best Price Pair Index 56, Prices: [2595 2595], Revenue: 305, Demand: 1
 Product 31044773: Best Price Pair Index 56, Prices: [1985 1985], Revenue: 315, Demand: 1
 Product 31044774: Best Price Pair Index 41, Prices: [1897 1897], Revenue: 0, Demand: 0
 Product 31044891: Best Price Pair Index 56, Prices: [732 732], Revenu

# new

# Adding INfeasibility Analysis

In [87]:
def initial_setup():
    folder_name= 'Experiments/July 26 Physical'
    model_names = ['CatBoostRegressor']
    date = '2024-07-26'

    inference_graphs_dir = f'{folder_name}/Inference_Graphs'  
    size = 100

    prices_list = []
    revenue_list = []
    cost_list = []
    demand_list = []
    product_ids_list = []
    national_price_creditA_list = []
    national_price_cashA_list = []
    national_price_DE_list = []
    margin_list = []
    min_credit_A_list = []
    max_credit_A_list = []
    price_segment_cluster_ls = []
    all_product_paths = []
    

    for model_name in model_names:
        base_path = os.path.join(inference_graphs_dir, model_name, '2024-07-26')
        if not os.path.exists(base_path):
            print(base_path)
            print("no")
            continue
        product_ids = os.listdir(base_path)
        for product_id in product_ids:
            try:  
                # float(product_id)
                product_dir = os.path.join(base_path, product_id)
                all_product_paths.append((product_id, product_dir))
            except ValueError:
                continue
        

    sorted_product_paths = sorted(all_product_paths)
    sorted_product_paths

    sku_map = pd.read_csv('data/sku_product_name_map.csv')
    national_price_df = pd.read_csv('data/national_price_20240722.csv',index_col=0)

    for product_id, product_dir in sorted_product_paths:
        # print(type(product_id))
        # print(product_id)
        csv_file = os.path.join(product_dir, f'{date}_uncertainty_metrics.csv')
        if os.path.exists(csv_file):
            
            df = pd.read_csv(csv_file)
            
            df = df[:size]
            
            df['Price Credit'] = df['Price Credit'].astype(int)
            df['Price Cash'] = df['Price Cash'].astype(int)
            df['Credit Prediction'] = df['Credit Prediction'].astype(int)
            df['Credit Pred Revenue'] = df['Credit Pred Revenue'].astype(int)
            # Convert pairs of 'Price Credit' and 'Price Cash' to lists
            price_pairs = [list(pair) for pair in zip(df['Price Credit'].values, df['Price Cash'].values)]
            prices_list.append(price_pairs)
            
            revenue = list(df['Credit Pred Revenue'].values)
            revenue = [int(value) for value in revenue]
            revenue_list.append(revenue)

            demand = list(df['Credit Prediction'].values)
            demand = [int(value) for value in demand]
            demand_list.append(demand)
            
            cost = list(df['Cost of Sale Credit'].values)
            cost = [abs(int(value)) for value in cost]
            cost_list.append(cost)
            
            product_ids_list.append(product_id)
            
            
            # print(len(national_price_df))
            # print('product',national_price_df[national_price_df['SKU']==str(product_id)])
            product_national_price = national_price_df[national_price_df['SKU']==int(product_id)].iloc[0]
            
            national_price_creditA = product_national_price['Control Credit Precio A']
            national_price_cashA = product_national_price['Control Cash Precio A']
            national_price_DE = product_national_price['Control Precio DE ($)']
            
            national_price_creditA_list.append(int(national_price_creditA))
            national_price_cashA_list.append(int(national_price_cashA))
            national_price_DE_list.append(int(national_price_DE))
            
            min_credit_A_list.append(int(df['Price Credit'].min()))
            max_credit_A_list.append(int(df['Price Credit'].max()))
            
            # price_segment = sku_map[sku_map['SKU']==product_id].iloc[0]
            price_segment_cluster_ls.append(product_national_price['price_segment'])

            margin = (df['Price Credit'].mean() - df['Cost of Sale Credit'].mean()) / df['Price Credit'].mean()
            margin_list.append(int(margin))  # Using mean margin for simplicity
        
            
            
            
            

    price_pair_options = [(i+1) for i in range(100)]
    # print(prices_list)

    price_array_3d = np.array([prices_list])

    prices_xr = xr.DataArray(prices_list, dims=["product", "price_pair_option", "price"], 
                                coords={"cluster":("product", price_segment_cluster_ls),
                                        "product": product_ids_list,
                                        "price_pair_option": price_pair_options
                                        }, name="Prices")
    demands_xr = xr.DataArray(demand_list, dims=["product", "price_pair_option"], 
                                coords={"product": product_ids_list, "price_pair_option": price_pair_options}, name="Demands")
    costs_xr = xr.DataArray(cost_list, dims=["product", "price_pair_option"], 
                            coords={"product": product_ids_list}, name="Costs")
    revenues_xr = xr.DataArray(revenue_list, dims=["product", "price_pair_option"], 
                                coords={"product": product_ids_list, "price_pair_option": price_pair_options}, name="Revenues")

    national_price_creditA_xr = xr.DataArray(national_price_creditA_list, dims=["product"], 
                                coords={"product": product_ids_list}, name="NationalPriceCreditA")
    
    national_price_cashA_xr = xr.DataArray(national_price_cashA_list, dims=["product"], 
                                coords={"product": product_ids_list}, name="NationalPriceCashA")

    national_price_DE_xr = xr.DataArray(national_price_DE_list, dims=["product"], 
                                coords={"product": product_ids_list}, name="NationalPriceDE")
    
    min_price_creditA_xr = xr.DataArray(min_credit_A_list, dims=["product"], 
                                coords={"product": product_ids_list}, name="MinPriceCreditA")
    max_price_creditA_xr = xr.DataArray(max_credit_A_list, dims=["product"], 
                                coords={"product": product_ids_list}, name="MaxPriceCreditA")

    margin_xr = xr.DataArray(margin_list, dims=["product"], 
                             coords={"product": product_ids_list}, name="Margins")
    
    return prices_xr, demands_xr, costs_xr, revenues_xr, national_price_creditA_xr, national_price_cashA_xr, national_price_DE_xr, min_price_creditA_xr, max_price_creditA_xr, margin_xr


def generate_data_xarray(num_clusters: int, num_products_per_cluster: list, num_price_pairs: int):
    np.random.seed(0)
    
    prices_list = []
    demands_list = []
    costs_list = []
    revenues_list = []
    
    cluster_coords = []
    product_coords = []
    
    product_index = 0
    for cluster_idx in range(num_clusters):
        num_products = num_products_per_cluster[cluster_idx]
        
        prices = np.random.randint(1000, 1300, size=(num_products, num_price_pairs, 2))
        demands = np.random.randint(2000, 10000, size=(num_products, num_price_pairs))
        costs = np.random.randint(200, 500, size=(num_products, num_price_pairs))
        revenues = np.random.randint(2000, 10000, size=(num_products, num_price_pairs))
        
        prices_list.append(prices)
        demands_list.append(demands)
        costs_list.append(costs)
        revenues_list.append(revenues)
        
        cluster_coords.extend([cluster_idx + 1] * num_products)
        product_coords.extend([(product_index + i + 1) for i in range(num_products)])
        product_index += num_products
    
    prices_combined = np.concatenate(prices_list, axis=0)
    demands_combined = np.concatenate(demands_list, axis=0)
    costs_combined = np.concatenate(costs_list)
    revenues_combined = np.concatenate(revenues_list, axis=0)
    
    price_pair_options = [(i+1) for i in range(num_price_pairs)]
    
    prices_xr = xr.DataArray(prices_combined, dims=["product", "price_pair_option", "price"], 
                             coords={"cluster": ("product", cluster_coords), "product": product_coords, "price_pair_option": price_pair_options}, name="Prices")
    demands_xr = xr.DataArray(demands_combined, dims=["product", "price_pair_option"], 
                              coords={"cluster": ("product", cluster_coords), "product": product_coords, "price_pair_option": price_pair_options}, name="Demands")
    costs_xr = xr.DataArray(costs_combined, dims=["product", "price_pair_option"], 
                            coords={"cluster": ("product", cluster_coords), "product": product_coords}, name="Costs")
    
    revenues_xr = xr.DataArray(revenues_combined, dims=["product", "price_pair_option"], 
                               coords={"cluster": ("product", cluster_coords), "product": product_coords, "price_pair_option": price_pair_options}, name="Revenues")
    
    return prices_xr, demands_xr, costs_xr, revenues_xr


def define_model(prices_xr, revenues_xr, costs_xr, demands_xr,national_price_creditA_xr, national_price_cashA_xr, national_price_DE_xr, margin_xr):
    num_products, num_price_pairs = prices_xr.shape[0], prices_xr.shape[1]
    price_indices = cp.intvar(0, num_price_pairs-1, shape=(num_products,), name='price_indices')
    model = cp.Model()

    add_revenue_objective(model, revenues_xr, price_indices)
    margin_constraints = add_constraints(model, prices_xr, costs_xr, price_indices, demands_xr, national_price_creditA_xr, national_price_cashA_xr, national_price_DE_xr, margin_xr)

    return model, price_indices, margin_constraints


def add_revenue_objective(model: cp.Model, revenues_xr, price_indices):
    total_revenue = sum(
        cp.Element(revenues_xr.values[p, :], price_indices[p])
        for p in range(revenues_xr.shape[0])
    )
    model.maximize(total_revenue)

def add_constraints(model: cp.Model, prices_xr, costs_xr, price_indices, demands_xr,national_price_creditA_xr, national_price_cashA_xr, national_price_DE_xr, margin_xr):
    
    total_cost_times_quantity = 0
    total_sale_price_times_quantity = 0
    
    for product in prices_xr.product.values:
        product_idx = prices_xr.get_index('product').get_loc(product)
        
        chosen_price = cp.Element(prices_xr.sel(product=product, price=0).values,
                              price_indices[product_idx])
        
        cost = cp.Element(costs_xr.sel(product=product).values,
                              price_indices[product_idx]) 
        quantity = cp.Element(demands_xr.sel(product=product).values,
                              price_indices[product_idx])  
        
        
        total_cost_times_quantity += cost * quantity
        total_sale_price_times_quantity += chosen_price * quantity
        
    # margin_constraint_upper = ((total_sale_price_times_quantity - total_cost_times_quantity) * 100 < 100 * total_sale_price_times_quantity)
    # model += margin_constraint_upper

    # margin_constraint_lower = ((total_sale_price_times_quantity - total_cost_times_quantity) * 100 > 1 * total_sale_price_times_quantity)
    # model += margin_constraint_lower
    
    for product in prices_xr.product.values:
        
        product_idx = prices_xr.get_index('product').get_loc(product)
        national_price_creditA = national_price_creditA_xr.sel(product=product).values
        national_price_DE  = national_price_DE_xr.sel(product=product).values
        
        model += (cp.Element(prices_xr.sel(product=product, price=0).values,
                              price_indices[product_idx]) * 100 <= national_price_creditA * 110
        )
        
        model += (cp.Element(prices_xr.sel(product=product, price=0).values,
                              price_indices[product_idx])* 100 >= national_price_creditA * 90
        )

        model += (cp.Element(prices_xr.sel(product=product, price=0).values,
                              price_indices[product_idx] )* 100  <=  national_price_DE * 95
        )
    
    # Group products by price segment
    margin_constraints = []
    
    # Group products by price segment
    price_segments = prices_xr.coords['cluster'].values
    unique_segments = np.unique(price_segments)

    for segment in unique_segments:
        segment_products = prices_xr.where(prices_xr.coords['cluster'] == segment, drop=True)
        segment_margins = margin_xr.where(prices_xr.coords['cluster'] == segment, drop=True)

        # Sort products by margin (descending order)
        sorted_indices = np.argsort(-segment_margins.values)
        sorted_products = segment_products.isel(product=sorted_indices)

        # Add constraints to ensure higher margin products have lower or equal prices
        for i in range(len(sorted_products.product) - 1):
            current_product = sorted_products.isel(product=i)
            next_product = sorted_products.isel(product=i + 1)

            current_product_index = int(np.where(prices_xr.product == current_product.product.values)[0][0])
            next_product_index = int(np.where(prices_xr.product == next_product.product.values)[0][0])

            current_price = cp.Element(prices_xr.sel(product=current_product.product.values, price=0).values, price_indices[current_product_index])
            next_price = cp.Element(prices_xr.sel(product=next_product.product.values, price=0).values, price_indices[next_product_index])

            constraint = (current_price <= next_price)
            model += constraint
            # Ensure we're storing scalar values for the product identifiers
            margin_constraints.append((constraint, current_product.product.values.item(), next_product.product.values.item()))

    return margin_constraints

def solve_model(model, price_indices, prices_xr, revenues_xr, demands_xr, national_price_creditA_xr, national_price_cashA_xr, min_price_creditA_xr, max_price_creditA_xr, national_price_DE_xr, margin_constraints):   
    ignored_products = set()
    
    while True:

        result = model.solve()
        
        if result:
            solution = pd.DataFrame(columns=['Product','Catalan Credit A','Catalan Cash A','National Credit A','National Cash A','National DE','Min Credit A','Max Credit A','Demand','Revenue'])

            
            print("Optimal total Revenue found:", model.objective_value())
            for p in range(prices_xr.shape[0]):
                price_index = price_indices[p].value()
                best_price_pair = prices_xr[p, price_index, :].values
                credit_A = prices_xr[p, price_index, 0].values
                cash_A = prices_xr[p, price_index, 1].values
                revenue = revenues_xr[p, price_index].values
                demand = demands_xr[p, price_index].values
                # cluster = prices_xr.coords['cluster'].values[p]
                product = prices_xr.coords['product'].values[p]
                natinal_credit_price = national_price_creditA_xr.sel(product=product).values
                natinal_cash_price = national_price_cashA_xr.sel(product=product).values
                min_credit_price = min_price_creditA_xr.sel(product=product).values
                max_credit_price = max_price_creditA_xr.sel(product=product).values
                national_DE_price = national_price_DE_xr.sel(product=product).values
                print(f" Product {product}: Best Price Pair Index {price_index}, Prices: {best_price_pair}, Revenue: {revenue}, Demand: {demand}")
            
                new_row = pd.DataFrame([[
                                            product,
                                            credit_A,
                                            cash_A,
                                            # best_price_pair,
                                            natinal_credit_price,
                                            natinal_cash_price,
                                            national_DE_price,
                                            min_credit_price,
                                            max_credit_price,
                                            demand,
                                            revenue
                                                ]], columns=['Product','Catalan Credit A','Catalan Cash A','National Credit A','National Cash A','National DE','Min Credit A','Max Credit A','Demand','Revenue'])
                solution = pd.concat([solution,new_row], ignore_index=True)
            
            solution.to_csv('Solution4.csv')
            
            
            if ignored_products:
                print(f"Solution found after ignoring these products for margin constraints: {ignored_products}")
            return solution
        else:
            print("No solution found. Attempting to identify and ignore problematic products for margin constraints...")
            ignored_product = find_problematic_product(model, margin_constraints)
            if ignored_product is None:
                print("Unable to identify a specific problematic product. Optimization failed.")
                return None
            
            # Ensure ignored_product is a scalar value before adding to the set
            ignored_product = ignored_product.item() if isinstance(ignored_product, np.ndarray) else ignored_product
            ignored_products.add(ignored_product)
            print(f"Ignoring product {ignored_product} for margin constraints and retrying optimization.")
            
            # Remove constraints involving the ignored product
            new_margin_constraints = []
            for constraint, prod1, prod2 in margin_constraints:
                # Ensure we're comparing scalar values
                prod1 = prod1.item() if isinstance(prod1, np.ndarray) else prod1
                prod2 = prod2.item() if isinstance(prod2, np.ndarray) else prod2
                if prod1 != ignored_product and prod2 != ignored_product:
                    new_margin_constraints.append((constraint, prod1, prod2))
                else:
                    model.remove(constraint)
            
            margin_constraints = new_margin_constraints

def find_problematic_product(model, margin_constraints):
    for constraint, prod1, prod2 in margin_constraints:
        if not constraint.value():
            # Return either product involved in the violated constraint
            # Ensure we're returning a scalar value, not an array
            return prod1.item() if isinstance(prod1, np.ndarray) else prod1
    return None



# prices_xr, demands_xr, costs_xr, revenues_xr = generate_data_xarray(num_clusters, num_products_per_cluster, num_price_options)
prices_xr, demands_xr, costs_xr, revenues_xr, national_price_creditA_xr, national_price_cashA_xr, national_price_DE_xr, min_price_creditA_xr, max_price_creditA_xr, margin_xr = initial_setup()
model, price_indices, margin_constraints = define_model(prices_xr, revenues_xr, costs_xr, demands_xr, national_price_creditA_xr, national_price_cashA_xr, national_price_DE_xr, margin_xr)
solution_df = solve_model(model, price_indices, prices_xr, revenues_xr, demands_xr, national_price_creditA_xr, national_price_cashA_xr, min_price_creditA_xr, max_price_creditA_xr, national_price_DE_xr, margin_constraints)

No solution found. Attempting to identify and ignore problematic products for margin constraints...
Ignoring product 31051965 for margin constraints and retrying optimization.


AttributeError: 'Model' object has no attribute 'remove'

In [5]:
def select_price_for_not_priced(not_priced, date):
    products = not_priced["SKU"].values
    products

    # national_price = pd.read_csv("data/national_price_06142024.csv", index_col=0)
    predictions = pd.DataFrame()
    for product_id in products:
        print(product_id)
        filtered_row = national_price[national_price["SKU"] == int(product_id)]
        print("filtered_row", filtered_row)
        credit_A_price = filtered_row["Control Credit Precio A"].iloc[0] * 0.97

        Cash_A_price = filtered_row["Control Cash Precio A"].iloc[0] * 0.97

        new_row = pd.DataFrame(
            [
                {
                    "Date": date,
                    "SKU": product_id,
                    "Best Price Credit A": credit_A_price,
                    "Best Price Cash A": Cash_A_price,
                    "Best Price Credit DE": filtered_row["Control Precio DE ($)"].iloc[0],
                    "Best Price Cash DE": filtered_row["Control Precio DE ($)"].iloc[0],
                    # "Credit Sales Prediction": 0,
                    # "Predicted Revenue": 0,
                    # "Cost of Sale Credit": filtered_row["Cost of Sale Credit"].iloc[0],
                    # 'Actual Credit Units Sold': optimal_row['Actual Credit Units Sold'],
                    # 'Actual Cash Price A': optimal_row['Actual Scaled Cash Price'],
                    # 'Actual Credit Price A': optimal_row['Actual Scaled Credit Price'],
                    # 'Actual Revenue': optimal_row['Actual Scaled Credit Price'] * optimal_row['Actual Credit Units Sold']
                }
            ]
        )

        numeric_columns = new_row.select_dtypes(include=["number"]).columns
        print(numeric_columns)
        # new_row[numeric_columns] = new_row[numeric_columns].round(0).astype(int)
        for col in numeric_columns:
            new_row[col] = new_row[col].astype(int)

        predictions = pd.concat([predictions, new_row])
    # predictions.to_csv("final_not_priced_june21.csv")
    return predictions

df = pd.read_csv('Solution4.csv',index_col=0)
df['SKU'] = df['Product']
df["Best Price Credit A"] = df['Catalan Credit A']
df["Best Price Cash A"] = df["Catalan Cash A"]
df["Best Price Credit DE"] = df["National DE"]
df["Best Price Cash DE"] = df["National DE"]
df["Date"] = ["2024-07-30"] * len(df)
df = df.drop(['Product','Catalan Credit A',"Catalan Cash A","National DE","Min Credit A","Max Credit A","Demand","Revenue","National Credit A","National Cash A"],axis=1)
df

sku_map = pd.read_csv('data/national_price_20240722.csv')
sku_map['SKU']

national_price = pd.read_csv('data/national_price_20240726.csv')

temp = sku_map.merge(df,on='SKU', how='outer', indicator=True)
only_in_df1 = temp[temp['_merge'] == 'left_only']

not_priced_solutions = select_price_for_not_priced(only_in_df1, '2024-07-30')
not_priced_solutions

combined = pd.concat([df, not_priced_solutions])
combined.to_csv("data/predictions/07302024_EKT_Physical_Stores_Prices_Final.csv")

31023696
filtered_row      Unnamed: 0      Estatus  Control Cash Precio A  Control Credit Precio A  \
506         506  Discontinuo                   1200                     1200   

     Control Precio DE ($)  Cost of Sale Credit  Margin       SKU  \
506                   3999              2757.93   0.125  31023696   

     price_segment  
506              1  
Index(['SKU', 'Best Price Credit A', 'Best Price Cash A',
       'Best Price Credit DE', 'Best Price Cash DE'],
      dtype='object')
31023697
filtered_row      Unnamed: 0      Estatus  Control Cash Precio A  Control Credit Precio A  \
505         505  Discontinuo                   1200                     1200   

     Control Precio DE ($)  Cost of Sale Credit  Margin       SKU  \
505                   3999              2757.93   0.125  31023697   

     price_segment  
505              1  
Index(['SKU', 'Best Price Credit A', 'Best Price Cash A',
       'Best Price Credit DE', 'Best Price Cash DE'],
      dtype='object')
3105